<a href="https://colab.research.google.com/github/kristynpantoja/math689project/blob/master/pytorchM2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install torch

In [2]:
!pip install torchvision

Model Parameters

In [0]:
# from types import SimpleNamespace

# args_dict = {"batch_size" : 50, "epochs" : 50, "no_cuda" : False, "seed" : 1, "log_interval" : 10}
# args = SimpleNamespace(**args_dict)
# args.epochs

50

In [5]:
!pip install gensim

    100% |████████████████████████████████| 23.6MB 1.6MB/s 
    100% |████████████████████████████████| 1.4MB 13.2MB/s 
    100% |████████████████████████████████| 133kB 24.8MB/s 
    100% |████████████████████████████████| 61kB 21.0MB/s 
    100% |████████████████████████████████| 4.7MB 5.8MB/s 
    100% |████████████████████████████████| 552kB 20.8MB/s 
  Running setup.py bdist_wheel for smart-open ... - done
  Stored in directory: /root/.cache/pip/wheels/23/00/44/e5b939f7a80c04e32297dbd6d96fa3065af89ecf57e2b5f89f
  Running setup.py bdist_wheel for bz2file ... - done
  Stored in directory: /root/.cache/pip/wheels/81/75/d6/e1317bf09bf1af5a30befc2a007869fa6e1f516b8f7c591cb9
Successfully built smart-open bz2file


Data: 20newsgroups
We get the document-term matrix

In [0]:
from __future__ import print_function
import argparse
import torch
import torch.utils.data
from torch import nn, optim
from torch.nn import functional as F
from torchvision import datasets, transforms
from torchvision.utils import save_image

from types import SimpleNamespace

from sklearn.datasets import fetch_20newsgroups
import numpy as np
import torch
# from gensim.models import Word2Vec
from sklearn.metrics.pairwise import cosine_similarity

from sklearn.feature_extraction.text import CountVectorizer
# categories = ['talk.politics.guns', 'sci.space', 'soc.religion.christian',
#               'misc.forsale', 'rec.sport.baseball', 'comp.sys.mac.hardware']
categories = ['talk.politics.guns', 'sci.space']
newsgroups_train = fetch_20newsgroups(subset='train', categories=categories)
vectorizer = CountVectorizer(stop_words = 'english', min_df=.01, max_df=0.9, 
                             token_pattern = u'(?ui)\\b[a-z]{3,}\\b')
count_vecs = vectorizer.fit_transform(newsgroups_train.data)
doc_term_matrix = count_vecs.toarray()
doc_term_matrix.shape # number of documents, number of words (in vocab)
tokenizer = vectorizer.build_tokenizer()

In [173]:
doc_term_matrix.shape[1] # vocab size

2441

In [174]:
len(vectorizer.get_feature_names())

2441

In [0]:
dict_word_freq = dict(zip(vectorizer.get_feature_names(), list(doc_term_matrix.sum(0))))

ragged array of words in each document (by index in vocabulary)

In [0]:
def countsToInput(row):
  return np.repeat(np.arange(doc_term_matrix.shape[1]),row)
  
def numWords(row):
  return row.sum()

N_train = np.apply_along_axis(numWords, axis=1, arr=doc_term_matrix)
data_train = []
for d in range(doc_term_matrix.shape[0]):
  data_train.append(torch.from_numpy(countsToInput(doc_term_matrix[d])))

## Word2vec

In [0]:
from gensim.models import Word2Vec, KeyedVectors


In [0]:
# ideally, we would do some preprocessing
newsgroups_train_preproc = []
for document in newsgroups_train.data:
    newsgroups_train_preproc.append(document.split())

# make the model
w2v = Word2Vec(sg=1, negative=5, size=100, window=10, min_count=1, max_vocab_size=None, max_final_vocab=None)
# w2v.build_vocab(newsgroups_train_preproc)
w2v.build_vocab_from_freq(word_freq = dict_word_freq)
# train the model
w2v.train(sentences=newsgroups_train_preproc,epochs=10, total_examples=doc_term_matrix.shape[1])
# save the model
w2v.save("sg1")

In [196]:
len(set([item for sublist in newsgroups_train_preproc for item in sublist]))

51319

In [197]:
torch.tensor(w2v.syn1neg).shape

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `syn1neg` (Attribute will be removed in 4.0.0, use self.trainables.syn1neg instead).
  """Entry point for launching an IPython kernel.


torch.Size([2441, 100])

In [198]:
len(w2v.wv.vocab)

2441

In [0]:
w2v.save("sg_1_M2")

In [199]:
w2v.syn1neg

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `syn1neg` (Attribute will be removed in 4.0.0, use self.trainables.syn1neg instead).
  """Entry point for launching an IPython kernel.


array([[-3.50045919e-01, -5.76119006e-01,  3.59132916e-01, ...,
        -9.80157554e-01,  6.87077761e-01, -1.63887873e-01],
       [-3.11047077e-01, -5.07865191e-01,  3.15256298e-01, ...,
        -8.64744723e-01,  6.06829166e-01, -1.45936355e-01],
       [-2.52017021e-01, -4.25064832e-01, -1.17316574e-01, ...,
        -4.85554546e-01,  1.96210504e-01,  2.83722520e-01],
       ...,
       [-6.67612478e-02, -2.71932453e-01, -1.09181866e-01, ...,
        -2.41017595e-01,  1.27573565e-01, -9.47324336e-02],
       [ 5.63658366e-04, -8.60928092e-03,  1.27539486e-01, ...,
        -3.16302866e-01,  7.16766790e-02,  4.46170047e-02],
       [-1.86148956e-01, -3.18686575e-01,  2.00770959e-01, ...,
        -5.40914536e-01,  3.77315015e-01, -8.76149833e-02]], dtype=float32)

In [200]:
w2v.wv.most_similar("university")

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('books', 0.9081634283065796),
 ('students', 0.9059314727783203),
 ('college', 0.9011502265930176),
 ('standard', 0.8874714374542236),
 ('page', 0.8783857822418213),
 ('library', 0.8773259520530701),
 ('atlas', 0.868262529373169),
 ('education', 0.8642970323562622),
 ('works', 0.8636724948883057),
 ('connection', 0.8621504306793213)]

## Setup for Model

In [0]:


# from types import SimpleNamespace


# args_dict = {"batch_size" : 50, "epochs" : 50, "no_cuda" : False, "seed" : 1, "log_interval" : 10}
# args = SimpleNamespace(**args_dict)
# args.cuda = not args.no_cuda and torch.cuda.is_available()


args_dict = {"batch_size" : 10, "epochs" : 50, "no_cuda" : False, "seed" : 1, "log_interval" : 100}
args = SimpleNamespace(**args_dict)


####


args.cuda = not args.no_cuda and torch.cuda.is_available()

torch.manual_seed(args.seed)

device = torch.device("cuda" if args.cuda else "cpu")

kwargs = {'num_workers': 1, 'pin_memory': True} if args.cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                   transform=transforms.ToTensor()),
    batch_size=args.batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.ToTensor()),
    batch_size=args.batch_size, shuffle=True, **kwargs)





Define model

In [0]:
class VAE(nn.Module):
    def __init__(self, num_docs):
        super(VAE, self).__init__()
        
        vocab_size = doc_term_matrix.shape[1]
        wordvec_dim = 100
        encoder_hidden = 256
        rp_normal_dim = 100
        K = 2
        self.num_docs = num_docs
#         self.word_embedding = nn.Embedding(vocab_size, wordvec_dim) # decoder
        self.word_embedding = nn.Embedding.from_pretrained(torch.tensor(w2v.syn1neg), freeze=True)
        self.topic_embedding = nn.Embedding(K, wordvec_dim) # decoder
        self.lin1 = nn.Linear(vocab_size, encoder_hidden) # encoder
        self.mean = nn.Linear(encoder_hidden, rp_normal_dim) # encoder
        self.logvar = nn.Linear(encoder_hidden, rp_normal_dim) # encoder
        self.lin2 = nn.Linear(rp_normal_dim, K) # decoder 
        #self.topicslayer = nn.Linear(wordvec_dim, K) # decoder
        self.beta = torch.zeros([K, vocab_size], dtype = torch.float32) # decoder
        #self.theta = torch.zeros([10, K], dtype = torch.float32)

    def encode(self, x):
        h1 = F.relu(self.lin1(x))
        return self.mean(h1), self.logvar(h1)

    def reparameterize(self, mu, logvar):
        std = torch.exp(0.5*logvar)
        eps = torch.randn_like(std)
        return eps.mul(std).add_(mu) # this gives x ~ N(mu, var)

      
    def decode(self, z):
        x = self.lin2(z) 
        theta = F.softmax(x, dim = 1) # to get theta, dim = batch size x K
        #word_dot_topic = self.topicslayer(self.word_embedding.weight) # weights corresp to topic vector
        self.beta = F.softmax(torch.mm(self.word_embedding.weight, 
                                       torch.transpose(self.topic_embedding.weight, 0, 1)), dim = 0) # beta, dim = V x K
        log_theta_dot_beta = torch.log(torch.mm(theta, torch.transpose(self.beta, 0, 1))) # dim = batch size x V
        #theta_dot_beta = torch.exp(log_theta_dot_beta - torch.logsumexp(log_theta_dot_beta, dim = 0))
        ####log_theta_dot_beta_normalized = log_theta_dot_beta - torch.logsumexp(log_theta_dot_beta, dim = 0)
        return log_theta_dot_beta ####log_theta_dot_beta_normalized
        
    def forward(self, doc):
        mu, logvar = self.encode(doc)
        z = self.reparameterize(mu, logvar)
        return self.decode(z), mu, logvar, self.topic_embedding.weight

Load training data (separate into batches)

In [0]:
# mnist_train_loader = torch.utils.data.DataLoader(
#     datasets.MNIST('../data', train=True, download=True,
#                    transform=transforms.ToTensor()),
#     batch_size=args.batch_size, shuffle=True, **kwargs)
# # enumerate(train_loader)

train_data = torch.utils.data.TensorDataset(torch.tensor(doc_term_matrix))
train_loader = torch.utils.data.DataLoader(train_data,                                            
    batch_size = args.batch_size, shuffle = True)

instantiate model and define functions for training

In [204]:
model = VAE(doc_term_matrix.shape[1]).to(device) 
      
optimizer = optim.Adam(model.parameters(), lr=1e-3)
# optimizer = optim.RMSprop(model.parameters(), lr = 1e-3)

# Reconstruction + KL divergence losses summed over all elements and batch
def loss_function(data, train_loader, log_theta_dot_beta, x, mu, logvar, t):
    BCE = data[0].shape[0] * 1.0 / len(train_loader.dataset) * log_theta_dot_beta.sum()
    KLD = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())
#     K = t.shape[0]
#     arccos = []
#     for j in range(K):
#         for k in range(j, K):
#             arccos.append(torch.acos(torch.dot(t[:, j], t[:, k]) /
#                                                    (max(t[:, j].norm() * t[:, k].norm(), 1e-5))))
#             arccos.append(F.cosine_similarity(t[:, j], t[:, k]))
#     arccos = torch.tensor(arccos)
#     print(arccos.max())
#     zeta = (1 / (K * K)) * arccos.sum()
#     nu = torch.zeros(1)
#     print("zeta: " + str(zeta) + "nu: " + str(nu))
#     for a in arccos:
#         nu = nu.add((a - zeta).pow(2))
#     nu = (1 / (K * K)) * nu
#     print("BCE: " + "{:.2f}".format(float(BCE)))
#     print("KLD: " + "{:.2f}".format(float(KLD)))
    return - BCE + KLD# - .1 * (zeta - nu)

enc_variables = list(model.lin1.parameters()) + list(model.mean.parameters()) +  list(model.logvar.parameters())
dec_variables = list(model.word_embedding.parameters()) + list(model.lin2.parameters()) + list(model.topic_embedding.parameters())


optim_enc = optim.Adam(enc_variables, lr=1e-3)
optim_dec = optim.Adam(dec_variables, lr=1e-3)

def train(epoch):
    model.train()
    train_loss = 0
    for switch in range(0,2):
        if switch == 0:
            print("updating encoder variables")
            optimizer = optim_enc
        else:
            print("updating decoder variables")
            optimizer = optim_dec
    for batch_idx, data in enumerate(train_loader):
        #data = data.to(device)
        optimizer.zero_grad()
        log_theta_beta, mu, logvar, topic_vecs = model(data[0].float())
        loss = loss_function(data, train_loader, log_theta_beta, data, mu, logvar, topic_vecs)
        loss.backward()
        train_loss += loss.item()
        optimizer.step()
        if batch_idx % args.log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * data[0].shape[0], len(train_loader.dataset),
                100. * batch_idx / len(train_loader),
                loss.item() / data[0].shape[0]))

    print('====> Epoch: {} Average loss: {:.4f}'.format(
          epoch, train_loss / len(train_loader.dataset)))

    return train_loss / len(train_loader.dataset)


# def test(epoch):
#     model.eval()
#     test_loss = 0
#     with torch.no_grad():
#         for i, (data, _) in enumerate(test_loader):
#             data = data.to(device)
#             recon_batch, mu, logvar = model(data)
#             test_loss += loss_function(recon_batch, data, mu, logvar).item()
#             if i == 0:
#                 n = min(data.size(0), 8)
#                 comparison = torch.cat([data[:n],
#                                       recon_batch.view(args.batch_size, 1, 28, 28)[:n]])
# #                 save_image(comparison.cpu(),
# #                          'results/reconstruction_' + str(epoch) + '.png', nrow=n)

#     test_loss /= len(test_loader.dataset)
#     print('====> Test set loss: {:.4f}'.format(test_loss))

# if __name__ == "__main__":
#     for epoch in range(1, args.epochs + 1):
#         train(epoch)
#         test(epoch)
#         with torch.no_grad():
#             sample = torch.randn(64, 20).to(device)
#             sample = model.decode(sample).cpu()
#             save_image(sample.view(64, 1, 28, 28),
#                        'results/sample_' + str(epoch) + '.png')

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:12: DeprecationWarning: Call to deprecated `syn1neg` (Attribute will be removed in 4.0.0, use self.trainables.syn1neg instead).
  if sys.path[0] == '':


 train the model

In [0]:
#model = VAE(doc_term_matrix.shape[1])#.to(device)

In [205]:
losses = []
for epoch in range(1, args.epochs + 1):
    losses.append(train(epoch))
#     if epoch > 1:
#         if np.abs(losses[epoch-1] - losses[epoch-2]) < 1e-2:
#             break


updating encoder variables
updating decoder variables
Train Epoch: 1 [0/1139 (0%)]	Loss: 191.855286
Train Epoch: 1 [1000/1139 (88%)]	Loss: 182.653345
====> Epoch: 1 Average loss: 186.2171
updating encoder variables
updating decoder variables
Train Epoch: 2 [0/1139 (0%)]	Loss: 181.181042
Train Epoch: 2 [1000/1139 (88%)]	Loss: 177.043335
====> Epoch: 2 Average loss: 179.1268
updating encoder variables
updating decoder variables
Train Epoch: 3 [0/1139 (0%)]	Loss: 176.910547
Train Epoch: 3 [1000/1139 (88%)]	Loss: 174.559924
====> Epoch: 3 Average loss: 175.8279
updating encoder variables
updating decoder variables
Train Epoch: 4 [0/1139 (0%)]	Loss: 174.318286
Train Epoch: 4 [1000/1139 (88%)]	Loss: 173.061731
====> Epoch: 4 Average loss: 173.8243
updating encoder variables
updating decoder variables
Train Epoch: 5 [0/1139 (0%)]	Loss: 172.814014
Train Epoch: 5 [1000/1139 (88%)]	Loss: 171.456128
====> Epoch: 5 Average loss: 172.4460
updating encoder variables
updating decoder variables
Train 

In [206]:
model.beta
_, ind = torch.sort(model.beta, 0)
# ind.numpy()[0:50, 0] - ind.numpy()[0:50, 1]
print(np.array(vectorizer.get_feature_names())[ind.numpy()][0:25, 0])
print(np.array(vectorizer.get_feature_names())[ind.numpy()][0:25, 1])
print(np.array(vectorizer.get_feature_names())[ind.numpy()][0:20, 2])
print(np.array(vectorizer.get_feature_names())[ind.numpy()][0:20, 3])
print(np.array(vectorizer.get_feature_names())[ind.numpy()][0:20, 4])
print(np.array(vectorizer.get_feature_names())[ind.numpy()][0:20, 5])
# print(np.array(vectorizer.get_feature_names())[ind.numpy()][0:20, 7])
# print(np.array(vectorizer.get_feature_names())[ind.numpy()][0:20, 8])
# print(np.array(vectorizer.get_feature_names())[ind.numpy()][0:20, 9])
# print(np.array(vectorizer.get_feature_names())[ind.numpy()][0:20, 15])
# print(np.array(vectorizer.get_feature_names())[ind.numpy()][0:20, 19])

['activity' 'added' 'agency' 'actual' 'company' 'exist' 'bar' 'created'
 'attempt' 'comments' 'better' 'armored' 'doug' 'afford' 'army' 'academic'
 'gsfc' 'hardly' 'stolen' 'cold' 'fully' 'dealing' 'criminals' 'ray'
 'ignore']
['additional' 'charles' 'affair' 'able' 'agents' 'actions' 'capable'
 'decade' 'advance' 'applications' 'cash' 'gap' 'automatic' 'act'
 'british' 'berkeley' 'bruce' 'deserve' 'finally' 'caught' 'dangerous'
 'effects' 'launcher' 'aurora' 'external']


IndexError: ignored

In [41]:
model.beta
torch.sort()

TypeError: ignored

In [15]:
# a = torch.randn(100, 128)
a = torch.tensor([[1,2,3], [4,5,6]]).float()
b = torch.tensor([[1,2,3], [4,5,6]]).float()

F.cosine_similarity(a, b)

tensor([1.0000, 1.0000])

Get word vectors and topic vectors

In [16]:
a = torch.tensor([1])
a.add(torch.tensor([2]))
print(a)

tensor([1])


In [0]:
for i in range(0, 2):
  print(i)

0
1


In [0]:
to

Parameter containing:
tensor([[ 1.0982, -0.6489,  0.0177,  ...,  0.7344,  0.6163,  0.6307],
        [-0.6461,  0.5238, -0.1179,  ..., -1.1260, -0.5322,  0.7694],
        [ 1.2182,  0.6254,  0.0470,  ...,  1.6700,  0.4856,  2.6060],
        ...,
        [ 0.5594,  1.2776,  1.0770,  ...,  0.8158,  1.5054, -0.4085],
        [ 1.0393, -0.1039,  0.9977,  ...,  1.5477, -0.1037,  0.0440],
        [ 0.7130,  1.8861, -1.6776,  ..., -0.1981,  1.5925, -1.3581]],
       requires_grad=True)

Get topic distributions

In [0]:
unscaled_topics = torch.mm(model.word_embedding(torch.tensor(np.arange(doc_term_matrix.shape[1]))),
         torch.transpose(model.topicslayer.weight, 0, 1))
topic_dist = torch.softmax(unscaled_topics, dim = 0)
topic_dist.sum(dim = 0)

tensor([1.0000, 1.0000, 1.0000, 1.0000], grad_fn=<SumBackward1>)

This one helped us a lot

In [0]:
#model.encode(torch.LongTensor(doc_term_matrix[0]))
#input = torch.tensor(doc_term_matrix).float()
input = torch.tensor(doc_term_matrix).float()[[0, 1], ]
mu, sigma = model.encode(input)
z = model.reparameterize(mu, sigma)
# model.decode(x, input.shape[0])

x = model.fc3(z)
theta = F.softmax(x) # to get theta
embedding_matrix = model.word_embedding(torch.tensor(np.arange(14)))
word_dot_topic = model.fc4(embedding_matrix) # weights corresp to topic vector
beta = F.softmax(word_dot_topic)
log_theta_dot_beta = torch.log(torch.mm(theta, torch.transpose(beta, 0, 1)))
#theta_dot_beta = torch.exp(log_theta_dot_beta - torch.logsumexp(log_theta_dot_beta, dim = 0))
log_theta_dot_beta_normalized = log_theta_dot_beta - torch.logsumexp(log_theta_dot_beta, dim = 0)
# print(theta.shape)
# print(theta)
# print(embedding_matrix)
# print(word_dot_topic)
print(beta.shape)
print(beta)
print(log_theta_dot_beta)
print(torch.exp(log_theta_dot_beta_normalized))


AttributeError: ignored

In [0]:
#model.encode(torch.LongTensor(doc_term_matrix[0]))
#input = torch.tensor(doc_term_matrix).float()
input = torch.tensor(doc_term_matrix).float()[[0, 1], ]
print(input)
mu, sigma = model.encode(input)
z = model.reparameterize(mu, sigma)
print(z)
# model.decode(x, input.shape[0])


# x = model.lin2(z)
# theta = F.softmax(x) # to get theta
# embedding_matrix = model.word_embedding(torch.tensor(np.arange(model.num_docs)))
# word_dot_topic = model.topicslayer(embedding_matrix) # weights corresp to topic vector
# model.beta = F.softmax(word_dot_topic, dim = 0)
# log_theta_dot_beta = torch.log(torch.mm(theta, torch.transpose(model.beta, 0, 1)))
# #theta_dot_beta = torch.exp(log_theta_dot_beta - torch.logsumexp(log_theta_dot_beta, dim = 0))
# log_theta_dot_beta_normalized = log_theta_dot_beta - torch.logsumexp(log_theta_dot_beta, dim = 0)
# print(embedding_matrix.shape) # dim of embedding matrix is 1544 x 100


x = model.lin2(z)
theta = F.softmax(x, 1) # to get theta
print(theta.sum(1))
embedding_matrix = model.word_embedding.weight
print(model.word_embedding(torch.tensor(np.arange(model.num_docs))).shape)
print(embedding_matrix.shape)
word_dot_topic = model.topicslayer(embedding_matrix) # weights corresp to topic vector
model.beta = F.softmax(word_dot_topic, dim = 0)
log_theta_dot_beta = torch.log(torch.mm(theta, torch.transpose(model.beta, 0, 1)))
#theta_dot_beta = torch.exp(log_theta_dot_beta - torch.logsumexp(log_theta_dot_beta, dim = 0))
log_theta_dot_beta_normalized = log_theta_dot_beta - torch.logsumexp(log_theta_dot_beta, dim = 0)
# print(embedding_matrix.shape) # dim of embedding matrix is still 1544 x 100


tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])
tensor([[-0.3542, -0.9824,  1.6668,  0.2909, -1.7727,  1.6447, -0.0335, -0.3574,
         -1.0229,  0.9524,  0.7662, -0.0573,  0.7221, -0.9453,  0.6689, -0.3142,
          1.1116, -0.5060, -0.5279,  0.1749,  0.5559, -0.7530, -0.2019, -0.1538,
          1.5552],
        [ 1.1045,  0.5738, -1.3214,  0.2868,  0.1640, -0.9203,  0.2256, -1.9079,
         -0.8485, -0.1582,  0.7142, -0.5238,  0.4466, -0.1109,  0.0060,  0.5935,
         -2.4257, -1.1120, -0.1708, -0.0022, -3.3380, -0.6230, -0.6414,  2.1400,
          0.5324]], grad_fn=<ThAddBackward>)
tensor([1.0000, 1.0000], grad_fn=<SumBackward1>)
torch.Size([1544, 100])
torch.Size([1544, 100])


In [0]:
from itertools import product

def topic_coherence(beta, M, doc_term_matrix):
  K = beta.shape[1] # beta has dim V x K
  coherences = np.zeros(K)
  for t in range(K):
    index = np.argsort(-beta[:, t])[0:M]
    cart_prod = product(list(index), list(index))
    for ind1, ind2 in cart_prod:
      if ind1 == ind2:
        pass
      else:
        d_ind1 = (doc_term_matrix[:, ind1] > 0).sum()
        d_ind12 = ((doc_term_matrix[:, ind1] > 0) & (doc_term_matrix[:, ind2] > 0)).sum()
        coherences[t] += np.log1p(d_ind12) - np.log(d_ind1)

  return coherences

In [145]:
topic_coherence(model.beta.detach().numpy(), 20, doc_term_matrix)

array([-989.91967797, -919.98065837])